In [1]:
%matplotlib inline

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import utils
import autoencoders

In [2]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


### Util functions

In [3]:
def plot_images(batch, nb_imgs):
    plt.figure(figsize=(20, 4))
    for i in range(nb_imgs):
        ax = plt.subplot(2, nb_imgs, i + 1)
        plt.imshow(batch[i].squeeze())
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

def plot_losses(losses, title='Evaluation losses'):
    plt.plot([i + 1 for i in range(len(losses))], losses, 'red')
    plt.title(title)
    plt.show()

def plot_results(mean_loss, batch_data, nb_imgs=9):
    (x, y, y_hat) = batch_data
    print(f'Mean loss: {round(mean_loss, 4)}')
    print('Input')
    plot_images(x, nb_imgs)
    print('Target')
    plot_images(y, nb_imgs)
    print('Generated')
    plot_images(y_hat, nb_imgs)

# Celeb_a

In [ ]:
(celeb_train, celeb_eval, celeb_test) = utils.load_dataset('celeb')

In [ ]:
encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64*64*3, activation='relu'),
    tf.keras.layers.Dense(32*32*3, activation='relu'),
])

decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(64*64*3, activation='sigmoid'),
    tf.keras.layers.Dense(218*178*3, activation='sigmoid'),
    tf.keras.layers.Reshape((218, 178, 3))
])

conv_encoder = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(218, 178, 3)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', strides=2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same', strides=2)
])

conv_decoder = tf.keras.Sequential([
    tf.keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=2, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(3, kernel_size=(3,3), activation='sigmoid', padding='same')
])

In [ ]:
default_celeb_cifar_autoencoder = autoencoders.Autoencoder(encoder, decoder)
eval_losses = default_celeb_cifar_autoencoder.fit(celeb_train, celeb_eval, epochs=20)
plot_losses(eval_losses)
plot_results(*default_celeb_cifar_autoencoder.evaluate(celeb_test))

In [ ]:
conv_celeb_cifar_autoencoder = autoencoders.Autoencoder(conv_encoder, conv_decoder)
eval_losses = conv_celeb_cifar_autoencoder.fit(celeb_train, celeb_eval, epochs=20)
plot_losses(eval_losses)
plot_results(*conv_celeb_cifar_autoencoder.evaluate(celeb_test))